## Question 1

In [1]:
from fastembed import TextEmbedding

In [2]:
import numpy as np

In [3]:
query = 'I just discovered the course. Can I join now?'
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
embedding = list(model.embed([query]))

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [4]:
print(embedding[0].shape)
print(np.min(embedding[0]))

(512,)
-0.11726373885183883


In [5]:
q = embedding[0]
print(np.linalg.norm(q))

1.0


In [6]:
print(q.dot(q))

1.0000000000000002


## Question 2

In [7]:
doc = 'Can I still join the course after the start date?'

In [8]:
embedded_doc = list(model.embed([doc]))
embedded_doc = embedded_doc[0]

In [9]:
print(np.dot(q, embedded_doc))

0.9008528895674548


## Question 3

In [10]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [11]:
texts = [doc['text'] for doc in documents]

In [12]:
emdedded_texts = list(model.embed(texts))

In [13]:
emdedded_texts = np.array(emdedded_texts)
print(emdedded_texts.shape)

(5, 512)


In [14]:
scores = emdedded_texts.dot(q)
print(scores.shape)
print(np.argmax(scores))
print(np.max(scores))

(5,)
1
0.8182378156620135


## Question 4

In [15]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [16]:
emdedded_full_texts = list(model.embed(full_texts))
emdedded_full_texts = np.array(emdedded_full_texts)
print(emdedded_full_texts.shape)

(5, 512)


In [17]:
scores = emdedded_full_texts.dot(q)
print(scores.shape)
print(np.argmax(scores))
print(np.max(scores))

(5,)
0
0.851454319443226


## Question 5

In [4]:
dimensions = [model["dim"] for model in TextEmbedding.list_supported_models()]
dimensions = np.array(dimensions)
min_dimensionality = np.min(dimensions)
print(f'minimal dimensionality : {min_dimensionality}')

minimal dimensionality : 384


In [5]:
dim_384_models = []
for model in TextEmbedding.list_supported_models():
    if model["dim"] == min_dimensionality:
        dim_384_models.append(model['model'])
if 'BAAI/bge-small-en' in dim_384_models:
    print(f'Model BAAI/bge-small-en has dimension {min_dimensionality}')

Model BAAI/bge-small-en has dimension 384


## Question 6

In [6]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
EMBEDDING_DIMENSIONALITY = min_dimensionality
model_handle = 'BAAI/bge-small-en'

In [8]:
collection_name = "machine-learning-zoomcamp-faq"

In [9]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

In [10]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [11]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [12]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model=model_handle 
    ),
    limit=5,
    with_payload=True
)

In [22]:
if len(query_points.points) > 0:
    print(f'max score is {query_points.points[0].score}')



max score is 0.87031734


In [23]:
query_points.points[0].payload

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp'}